In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
import random
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19/dataset.xlsx


In [2]:
SEED = 2019
random.seed(SEED)
np.random.seed(SEED)

# **Data Loading and Preprocessing**

In [3]:
df = pd.read_excel('/kaggle/input/covid19/dataset.xlsx')

In [4]:
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['SARS-Cov-2 exam result'] = [0 if a == 'negative' else 1 for a in df['SARS-Cov-2 exam result'].values]

Y = df['SARS-Cov-2 exam result']

df = df.drop([
    "SARS-Cov-2 exam result",
    "Patient ID",
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)'
], axis=1)

In [6]:
df = df.fillna(df.mean())

In [7]:
# Fill NaNs with -10
df = df.fillna(-10, axis=1)

In [8]:
categorical_features = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['object'] ]

In [9]:
X = pd.get_dummies(df, prefix=categorical_features, columns=categorical_features)

# RandomForestClassifier

clf = RandomForestClassifier(max_depth=50, random_state=0,n_estimators=1000)

In [10]:
clf = RandomForestClassifier(max_depth=50, random_state=0,n_estimators=1000)
K = 10
folds = KFold(K, shuffle = True, random_state = SEED)

for fold , (train_index,test_index) in enumerate(folds.split(X, Y)):
    print('Fold:',fold+1)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    
    clf.fit(X_train,y_train)

    
    pred_y = clf.predict(X_test)
   
    
    print(classification_report(y_test,pred_y))

Fold: 1
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       512
           1       0.00      0.00      0.00        53

    accuracy                           0.90       565
   macro avg       0.45      0.50      0.47       565
weighted avg       0.82      0.90      0.86       565

Fold: 2
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       507
           1       0.50      0.02      0.03        58

    accuracy                           0.90       565
   macro avg       0.70      0.51      0.49       565
weighted avg       0.86      0.90      0.85       565

Fold: 3
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       510
           1       1.00      0.02      0.04        55

    accuracy                           0.90       565
   macro avg       0.95      0.51      0.49       565
weighted avg       0.91      0.90      0.86       

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95       510
           1       0.00      0.00      0.00        54

    accuracy                           0.90       564
   macro avg       0.45      0.50      0.47       564
weighted avg       0.82      0.90      0.86       564

Fold: 7
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       507
           1       1.00      0.02      0.03        57

    accuracy                           0.90       564
   macro avg       0.95      0.51      0.49       564
weighted avg       0.91      0.90      0.86       564

Fold: 8
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       499
           1       0.25      0.02      0.03        65

    accuracy                           0.88       564
   macro avg       0.57      0.50      0.48       564
weighted avg       0.81      0.88      0.83       564

Fol

In [11]:
#Acurracy 0.90 but recall for the class "positive" close to zero or zero